# Human-Action-Recognition-VIT-Base-patch16-224

This model is a fine-tuned version of google/vit-base-patch16-224 on Bingsu/Human_Action_Recognition dataset.

https://huggingface.co/rvv-karma/Human-Action-Recognition-VIT-Base-patch16-224

Has 15 classes: calling, clapping, cycling, dancing, drinking, eating, fighting, hugging, laughing, listening_to_music, running, sitting, sleeping, texting, using_laptop

-> NOT INCLUDING WALKING -> MAY NEED TO FIND ANOTHER MODEL

In [1]:
from PIL import Image
import requests
from transformers import pipeline

In [2]:
pipe = pipeline("image-classification", "rvv-karma/Human-Action-Recognition-VIT-Base-patch16-224")
url = "https://images.pexels.com/photos/175658/pexels-photo-175658.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500"
image = Image.open(requests.get(url, stream=True).raw)
pipe(image)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

[{'label': 'Dancing', 'score': 0.9918228983879089},
 {'label': 'Clapping', 'score': 0.0022566637489944696},
 {'label': 'Running', 'score': 0.0014649435179308057},
 {'label': 'Fighting', 'score': 0.0008779025520198047},
 {'label': 'Sitting', 'score': 0.0007207187591120601}]

In [4]:
from IPython.display import clear_output, display, Image
import os
import subprocess
import pandas as pd
import re
import cv2
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
video_path = "/content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/video/sample_video_pop_estimation.mp4"

output_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/content", "extracted_frames")
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
  print("Error opening video!")
  exit()

# frames per second
fps = int(cap.get(cv2.CAP_PROP_FPS))
# total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_count = 1
actual_count = 0

# Read until video is completed
while True:
  ret, frame = cap.read()

  if not ret:
    print("Can't receive frame (stream end?). Exiting...")
    break

  # Save frame as numbered image
  if frame_count % fps == 0:
    image_path = os.path.join(output_dir, f"{frame_count}.jpg")
    cv2.imwrite(image_path, frame)
    actual_count += 1

  frame_count += 1

  if frame_count > total_frames:
    break

cap.release()

print(f"Extracted {actual_count} frames to {output_dir}")

Extracted 308 frames to /content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/content/extracted_frames


In [12]:
# Path to your extracted frames directory
frames_dir = "/content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/content/extracted_frames"

# Get list of image files in the directory
image_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.jpg')])

In [10]:
"""from PIL import Image
# Path to the image file you want to process
image_path = "/content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/content/extracted_frames/112.jpg"

# Open the image
image = Image.open(image_path)
pipe(image)

[{'label': 'Running', 'score': 0.4390575885772705},
 {'label': 'Fighting', 'score': 0.4084160029888153},
 {'label': 'Dancing', 'score': 0.04691242054104805},
 {'label': 'Sitting', 'score': 0.03396705165505409},
 {'label': 'Sleeping', 'score': 0.020511724054813385}]

In [14]:
from PIL import Image

# Create empty lists to store results
image_names = []
labels = []
scores = []

# Process each image using the pipeline
for image_file in image_files:
    image = Image.open(image_file)
    result = pipe(image)

    # Extract label and score from the result
    label = result[0]['label']
    score = result[0]['score']

    # Store image filename, label, and score
    image_names.append(os.path.basename(image_file))
    labels.append(label)
    scores.append(score)

# Create a DataFrame from the results
df = pd.DataFrame({
    'Image_File': image_names,
    'Label': labels,
    'Score': scores
})

,Image_File,Label,Score
0,1008.jpg,Sitting,0.199696
1,1022.jpg,Running,0.319549
2,1036.jpg,Running,0.298702
3,1050.jpg,Running,0.292977
4,1064.jpg,Running,0.520808
...,...,...,...
303,952.jpg,Fighting,0.189792
304,966.jpg,Running,0.220632
305,98.jpg,Running,0.564882
306,980.jpg,Running,0.229555


In [15]:
df

,Image_File,Label,Score
0,1008.jpg,Sitting,0.199696
1,1022.jpg,Running,0.319549
2,1036.jpg,Running,0.298702
3,1050.jpg,Running,0.292977
4,1064.jpg,Running,0.520808
...,...,...,...
303,952.jpg,Fighting,0.189792
304,966.jpg,Running,0.220632
305,98.jpg,Running,0.564882
306,980.jpg,Running,0.229555


In [17]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Visualization for ML/Project/results.csv", index=False)